# 2. Get the Anecdote to Estimate Impact

Notebook to build all of the different stix objects associated with asking the reporting user about the Impact, and recording it through an Anecdote

We take v3.0 of the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc) specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are nine stages:

1. First setup the global parameters and retrieve the context memory
2. Create the Anecdote object and connect it to the Identity,
2. Add these objects to the `ObservedData` object
4. Create the `Sighting` object with the `SightingAnecdote` extension
5. Create the Impact object
5. Create the next Task and its Sequence objects
6. Connect the Sequence object to the previous sequence object
6. Add all of the objects to the existing Incident object
7. Save the context memory


Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).



## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [3]:
import json
from Block_Families.Objects.SCO.Email_Message.make_email_msg import main as make_email_msg
from Block_Families.Objects.SRO.Relationship.make_sro import main as make_sro
from Utilities.local_make_general import invoke_context_get_block, invoke_context_save_block
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block, invoke_make_incident_block, invoke_make_impact_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block, invoke_make_anecdote_block
)
from conv import conv

context_base = "../Orchestration/Context_Mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
anecdote_data = {
    "to": "naive@example.com"
}

TR_Context_Memory_Path = "./Context_Mem/Type_Refinery_Context.json"

### A.4 Load the Context Memory

In [4]:
# 1. Load the Context
Type_Refinery_Context = invoke_context_get_block()
#
# 2. Setup the  TR User Context
#
local = Type_Refinery_Context["local"]
me = local["me"]
team = local["team"]
company = local["company"]
systems = local["systems"]
assets = local["assets"]
#
# 3. Setup the Incident Context
#
incident = Type_Refinery_Context["incident"]
sequence_start_objs = incident["sequence_start_objs"]
sequence_objs = incident["sequence_objs"]
task_objs = incident["task_objs"]
event_objs = incident["event_objs"]
impact_objs = incident["impact_objs"]
other_object_objs = incident["other_object_objs"]
incident_obj = incident["incident_obj"]

### A.5 Retrieve the TR User Account, the Email Address, and the TR Identity from the Context Memory

and the previous task sequence

In [5]:
# 1. For Naive Smith, find the user account and the email. Plus get my own identity
to_email_addr = anecdote_data["to"]
TR_name = "Trusty Jones"
# 2. Setup variables to hold the dicts
reporter_email_addr = {}
reporter_identity = {}
TR_identity = {}
reporter_laptop = {}
#
# 3. Retrieve the details for the employees involved
#
for rec in company:
    if rec["type"] == "email-addr":
        if rec["value"] == to_email_addr:
            reporter_email_addr = rec

for rec in company:
    if rec["type"] == "identity" and rec["identity_class"] == "individual":
        for email_address in rec["extensions"]["extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498"]["email_addresses"]:
            if email_address["email_address_ref"] == reporter_email_addr["id"]:
                reporter_identity = rec

for rec in me:
    if rec["type"] == "identity":
        TR_identity = rec

for rec in assets:
    if rec["name"] == "HP Laptop 1":
        reporter_laptop = rec


## B Make the Anecdote

In [6]:
# 1. Setup the form paths and the results path
anecdote_form = "SCO/Anecdote/anecdote_on_impact.json"
results_path = "step2/anecdote.json"
# 2. Invoke the Make Anecdote block
anecdote_list = invoke_make_anecdote_block(anecdote_form, results_path, anecdote_reporter=reporter_identity)
# 3. Add the record to the in-session bundles and lists
other_object_objs.append(anecdote_list[0])
# recover the Anecdote object
anecdote = conv(anecdote_list[0])

{
    "type": "anecdote",
    "spec_version": "2.1",
    "id": "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f",
    "value": "I clicked on the link, and my laptop screen went weird",
    "report_date": "2023-12-31T08:03:28.837467Z",
    "provided_by_ref": "identity--e5a6e5c3-7138-4288-8146-87d972eb2098"
}


## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [7]:
# 1. Setup0 list of obsrvations
observed_ids = []
observed_ids.append(anecdote["id"])
# 2. Setup path to form and results
obs_path ="SDO/Observed_Data/observation-alert.json"
results_path ="step1/observation-anecdote.json"
# 3. Invoke the Make Observed Data Block
obs_2 = invoke_make_observed_data_block(obs_path, results_path, observation=observed_ids)
# 4. Add the record to the in-session bundles and lists
other_object_objs.append(obs_2[0])
# recover the observation object
observe_2 = conv(obs_2[0])

{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--2a710640-d225-452f-a454-f110fa5c4c9e",
    "created": "2023-12-31T08:03:28.864646Z",
    "modified": "2023-12-31T08:03:28.864646Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f"
    ]
}


## D. Create the Anecdote Sighting

Tbis then represents the Anecdote on Impact from the reporting user

### D.1 Create the Sighting
Connecting the:
- observed-data object
- the reporter identity as the sighted object
- the laptop identity as the location

In [8]:
# 1. Setup path to form and results
sighting_data_path ="SRO/Sighting/sighting_anecdotet.json"
results_path ="step2/sighting_anecdote.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing #
observed_list = [observe_2["id"]]
sighted = reporter_identity["id"]
TR_where_list = [TR_identity["id"]]
# 2. Invoke the Make Observed Data Block
sight2 = invoke_sighting_block(sighting_data_path, results_path, observed=observed_list, sighted=sighted, where=TR_where_list)
# 3. Add the record to the in-session bundles and lists
other_object_objs.append(sight2[0])

{
    "type": "sighting",
    "spec_version": "2.1",
    "id": "sighting--fd535c1c-a12b-4e12-8d62-e99c752b8cdf",
    "created": "2023-12-31T08:03:28.901047Z",
    "modified": "2023-12-31T08:03:28.901047Z",
    "count": 1,
    "sighting_of_ref": "identity--e5a6e5c3-7138-4288-8146-87d972eb2098",
    "observed_data_refs": [
        "observed-data--2a710640-d225-452f-a454-f110fa5c4c9e"
    ],
    "where_sighted_refs": [
        "identity--2e02a834-c89b-483b-869d-fcb713721246"
    ],
    "extensions": {
        "extension-definition--0d76d6d9-16ca-43fd-bd41-4f800ba8fc43": {
            "extension_type": "property-extension"
        },
        "sighting-anecdote": {
            "person_name": "user-report",
            "person_context": "employee",
            "report_submission": "interview"
        }
    }
}


## E. Create the Impact Objects

Tbis Impact represents the effect reported by the user in the Anecdote

In [9]:
# 1. Setup path to form and results
impact_path ="SDO/Impact/anecdote_impact.json"
results_path ="step2/impact_anecdote.json"
# 2. Setup the number of assets impacted
numbers = {"computers-mobile": 1}
impacted = reporter_laptop["id"]
# 2. Invoke the Make Observed Data Block
impact_1 = invoke_make_impact_block(impact_path, results_path, impacted_entity_counts=numbers, impacted_refs=impacted, superseded_by_ref=None)
# 3. Add the record to the in-session bundles and lists
impact_objs.append(impact_1[0])
other_object_objs.append(reporter_laptop)

{
    "type": "impact",
    "spec_version": "2.1",
    "id": "impact--dfb745f7-a931-4f3a-a6ef-6e9f8ae4a018",
    "created": "2023-12-31T08:03:28.94749Z",
    "modified": "2023-12-31T08:03:28.94749Z",
    "impact_category": "availability",
    "criticality": 99,
    "description": "Laptop is stuffed",
    "end_time": "2023-12-31T08:03:28.94749Z",
    "impacted_entity_counts": {
        "computers-mobile": 1
    },
    "impacted_refs": [
        "identity--75d7bcb7-8639-45f4-952e-96845b95e109"
    ],
    "recoverability": "regular",
    "start_time": "2023-12-31T08:03:28.94749Z",
    "extensions": {
        "extension-definition--7cc33dd6-f6a1-489b-98ea-522d351d71b9": {
            "extension_type": "new-sdo"
        }
    }
}


## F. Create the Next Task Object

Next step is to investigate the Exchange server to see who else got the email

### F.1 Create the Task Object

In [10]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_anecdote.json"
results_path ="step2/task_anecdote.json"
# 2. Invoke the Make Observed Data Block
task_2 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
task_objs.append(task_2[0])

{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--3d8b10a7-0217-4784-a17d-26f55a68da0a",
    "created": "2023-12-31T08:03:28.978507Z",
    "modified": "2023-12-31T08:03:28.978507Z",
    "description": "Find out who else got the email",
    "end_time": "2023-12-31T08:03:28.978507Z",
    "name": "Query Exchange Server",
    "start_time": "2023-12-31T08:03:28.978507Z",
    "extensions": {
        "extension-definition--2074a052-8be4-4932-849e-f5e7798e0030": {
            "extension_type": "new-sdo"
        }
    }
}


### F.2 Create the Sequence Object

In [11]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_task_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_2[0]["id"]
# 2. Invoke the Make Observed Data Block
seq_T_2 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
sequence_objs.append(seq_T_2[0])

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--27d43ed5-0a50-47da-9ca5-2df81f9873be",
    "created": "2023-12-31T08:03:29.010237Z",
    "modified": "2023-12-31T08:03:29.010237Z",
    "sequenced_object": "task--3d8b10a7-0217-4784-a17d-26f55a68da0a",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--4ca6de00-5b0d-45ef-a1dc-ea7279ea910e": {
            "extension_type": "new-sdo"
        }
    }
}


### F.3 Connect the Sequence to the Previous one

In [12]:
# Find the Previous Sequence and link it to the above Sequence
last_task = task_objs[-1]
for rec in sequence_objs:
    if rec["sequence_type"] == "task" and rec["sequenced_object"] == last_task["id"]:
        rec["on_completion"] = seq_T_2[0]["id"]

## G. Finally, Append all of the objects to the Incident Object

Assign all of the objets to the Incident Object

In [13]:
# 1. Setup path to form and results
inc_path ="SDO/Incident/phishing_incident.json"
results_path ="step1/incident_alert.json"
# 2. Setup the Sequence Object for the Event
#
sequence_start_refs = [x["id"] for x in sequence_start_objs]
sequence_refs = [x["id"] for x in sequence_objs]
task_refs = [x["id"] for x in task_objs]
event_refs = [x["id"] for x in event_objs]
impact_refs = [x["id"] for x in impact_objs]
other_object_refs = [x["id"] for x in other_object_objs]
# 3. Update the actual Incident Object
incident_ext = incident_obj["extensions"]["extension-definition--ef765651-680c-498d-9894-99799f2fa126"]
incident_ext["sequence_start_refs"] = sequence_start_refs
incident_ext["sequence_refs"] = sequence_refs
incident_ext["task_refs"] = task_refs
incident_ext["event_refs"] = event_refs
incident_ext["impact_refs"] = impact_refs
incident_ext["other_object_refs"] = other_object_refs

## H. Write out the Contex Memory for the Incident

Export out the Context Memory for the Incident

In [14]:
# Save the Tpe Refinery Context Memory File
invoke_context_save_block(Type_Refinery_Context)